In [1]:
import pickle
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import numpy as np
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [2]:
def invisible(ax):
    # ax=plt.gca()  #gca:get current axis得到当前轴
    #设置图片的右边框和上边框为不显示
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['bottom'].set_color('none')
    ax.axes.get_yaxis().set_visible(False)
    # x 轴不可见
    ax.axes.get_xaxis().set_visible(False)

In [3]:
dir = 'citation1'
# dir = 'weibo1'
valid = pickle.load(open("/data/cas2vec/"+dir+"/valid_ana.pkl",'rb'))
test = pickle.load(open("/data/cas2vec/"+dir+"/test_ana.pkl",'rb'))
valid.keys(),test.keys()

(dict_keys(['outputs_prediction', 'outputs_true', 'graph_hidden', 'rnn_hidden', 'graph_info', 'ptr', 'nodes_f', 'nodes_map']),
 dict_keys(['outputs_prediction', 'outputs_true', 'graph_hidden', 'rnn_hidden', 'graph_info', 'ptr', 'nodes_f', 'nodes_map']))

In [4]:
nodes_f = test['nodes_f']+valid['nodes_f']
nodes_map = test['nodes_map']+valid['nodes_map']
ptr = np.concatenate((test['ptr'],valid['ptr']))
test_trues = np.concatenate((test['outputs_true'],valid['outputs_true']))
test_graph_size = []
test_graph_hidden = np.concatenate((test['graph_hidden'],valid['graph_hidden']))
test_rnn_hidden = np.concatenate((test['rnn_hidden'],valid['rnn_hidden']))
graph_info = test['graph_info']+valid['graph_info']
n_seq = test['graph_hidden'].shape[2]
hidden_dim = test['graph_hidden'].shape[-1]

for iter in ptr:
    for idx in range(0,len(iter)-1,n_seq):
        one_batch = iter[idx+1:idx+n_seq+1]-iter[idx:idx+n_seq]
        test_graph_size.append(one_batch)
# test_graph_size=np.array(test_graph_size)
test_graph_size=np.log2(np.array(test_graph_size)+1)
# print(test_graph_size.shape)
last_moment = test_graph_size.T[-1]
# print(last_moment)
# print(np.expand_dims(test_graph_size.T[-1],axis=1)-test_graph_size)

In [5]:
from tqdm import tqdm
import time


# print(os.getcwd())


def get_hour(time_str, filename):
    hour = None
    try:
        msg_time = int(time_str)
        hour = time.strftime("%H", time.localtime(msg_time))
        hour = int(hour)
    except:
        if '170w' in filename:  # fixed in 11.15, however, in this way, more datasets will be removed
            ts = time.strptime(time_str, '%Y-%m-%d-%H:%M:%S')
            hour = ts.tm_hour
        elif 'castle' in filename:
            # for data castle weibo
            hour = int(time_str[:2])
        elif 'smp' in filename:
            ts = time.strptime(time_str, '%Y-%m-%dT%H:%M:%S')
            hour = ts.tm_hour
        else:
            print(time_str)
            print('wrong time format')
    return hour
import networkx as nx
def construct_G(filename='/home/ta/yh/open-cas/rawdata/dataset_weibo.txt'):
    G = nx.Graph()
    cascade_count={}
    # filename='./dataset_citation.txt'
    discard_midnight = 0
    discard_outer = 0       
    file = open(filename)
    cascades_total = dict()
    year_grow = dict()
    num_lines = sum(1 for line in open(filename, 'r'))
    for line in tqdm(file, total=num_lines):
        parts = line.strip().split("\t")
        if len(parts) != 5:
            print('wrong format!')
            continue
        cascadeID = parts[0]
        # print cascadeID
        n_nodes = int(parts[3])

        path = parts[4].split(" ")
        if n_nodes != len(path):
            print(cascadeID,' wrong number of nodes', n_nodes, len(path),path)
            exit(0)
        # for p in path:
        #     nodes = p.split(":")[0].split("/")
        #     time_ = int(int(p.split(":")[1]))
        #     if time_<=3600:
        #         num+=1
        # if num>100 :
        #     discard_outer+=1
        #     continue
        # if n_nodes<5 or n_nodes>100:
        #     continue
        hour = get_hour(parts[2], filename)
        # to keep the same with
        if hour <= 7 or hour >= 19:  # 8-18
            discard_midnight += 1
            continue
        observation_path = []
        # print(cascadeID)
        for p in path:
            nodes = p.split(":")[0].split("/")
            time_ = int(int(p.split(":")[1])/360)
            try:
                year_grow[time_]+=1
            except:
                year_grow[time_]=1
            nodes_ok = True
            for n in nodes:
                if int(n) == -1:  # delete invalid id
                    nodes_ok = False
            if not (nodes_ok):
                print("error id at cas_id {}".format(cascadeID))
                print(nodes)
                continue
            # print nodes
            time_now = int(p.split(":")[1])

            observation_path.append(",".join(nodes) + ":" + str(time_now))
            for i in range(1, len(nodes)):
                G.add_edge(nodes[i - 1] , nodes[i])
        # if len(observation_path)<5 or  len(observation_path) > 1000:
        #     # least_rewteetnum = opts.least_num   # 5 or 10
        #     # if len(observation_path) < least_rewteetnum:
            
        #     discard_outer += 1
        #     continue


        # if labels[0]>1000:
        #     continue


        # try:
        #     cascades_total[cascadeID] = msg_time
        # except:
        cascades_total[cascadeID] = hour

    n_total = len(cascades_total)
    print("total_readin:", num_lines)
    print("discard_midnight:", discard_midnight)
    print("discard_outer:", discard_outer)
    print('total:', n_total)
    # import operator
    # sorted_msg_time = sorted(cascades_total.items(),
    #                          key=operator.itemgetter(1))
    # 计算每个节点的 PR 值，并作为节点的 pagerank 属性
    print('computing pange rank')
    pagerank = nx.pagerank(G)
    # 将 pagerank 数值作为节点的属性
    nx.set_node_attributes(G, name = 'pagerank', values=pagerank)
    print('computing done')

    return G
    # 划分数据集



In [6]:
G = construct_G('/home/ta/yh/open-cas/rawdata/dataset_citation.txt')

100%|██████████| 72936/72936 [00:18<00:00, 4006.33it/s]


total_readin: 72936
discard_midnight: 0
discard_outer: 0
total: 72936
computing pange rank
computing done


In [7]:
list(G.nodes())[:10]

['855', '861', '873', '866', '856', '868', '864', '862', '877', '865']

In [10]:
page_rank = nx.get_node_attributes(G, "pagerank")
# page_rank['1130594']

In [11]:
# nodes_f = test['nodes_f']+valid['nodes_f']
# nodes_map = test['nodes_map']+valid['nodes_map']
node2f = {}
for batch_f,batch_map in zip(nodes_f,nodes_map):
    for index,node in  batch_map.items():
        try:
            node2f[node].append(batch_f[index])
        except:
            node2f[node]=[batch_f[index]]
# id2index={}
# for 

In [ ]:
len(node2f),len(G)

(279862, 4818928)

In [129]:
X = []
y = []
import random
for k,v in node2f.items():
    y.append(page_rank[k])
    X.append(np.array(node2f[k][-1]))
    # X.append(np.array(random.choice(node2f[k])))
y=np.array(y)
X=np.array(X)
X.shape

(127504, 32)

In [139]:
avg = np.average(y)
top = sorted(y,reverse=True)[int(len(y)*0.01)]
sample = int(len(y)*0.01)
print(sample)
print(avg)
label = []
new_X = []
for i,x in zip(y,X):
    if i>top:
        label.append(1)
        new_X.append(x)
    # elif i > avg*0.9:
    #     label.append(1)
    elif sample > 0:
        sample-=1
        label.append(-1)
        new_X.append(x)
# X label
label=np.array(label)
np.sum(label==0),np.sum(label==1),np.sum(label==-1)

1275
4.298437295985787e-06


(0, 1275, 1275)

In [141]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_X, label,test_size=0.25, stratify = label)
clf = MLPClassifier(random_state=1).fit(X_train, y_train)
clf.predict_proba(X_test)
clf.predict(X_test)
mlp_predictions = clf.predict(X_test)
cm = confusion_matrix(y_test, mlp_predictions)
clf.score(X_test, y_test)

/home/ta/.local/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.5815047021943573

In [36]:
print(cm)

[[215 104]
 [201 118]]


In [ ]:
len(graph_info),len(test_graph_hidden),len(graph_info[0]),len(test_graph_hidden[0]),test_graph_hidden.shape

In [ ]:
len(nodes_f),len(nodes_f[2])

In [ ]:
print(np.max(graph_info[2][0]))

In [ ]:
test['graph_hidden'].shape,valid['graph_hidden'].shape

In [ ]:
output_pool = []
for outputs in test_graph_hidden:
    outputs = torch.tensor(outputs)
    lstm_output = F.adaptive_max_pool1d(outputs.transpose(1, 2), output_size=1).squeeze()
    output_pool.append(lstm_output.tolist())
output_pool = np.array(output_pool)
output_pool.shape

## graph_hidden-graph_size

In [ ]:
from sklearn.utils import shuffle
print(test_graph_size.shape,test_graph_hidden.shape)
plt.close()
invisible(plt.gca())
Hs,Is = shuffle(test_graph_hidden.reshape(-1,hidden_dim),np.power(2,test_graph_size).reshape(-1))
X = TSNE(n_components=2,random_state=42,learning_rate=25,n_iter=6000,verbose=1).fit_transform(Hs[:80000])
sc = plt.scatter(X[:, 0], X[:, 1],s=1, c=Is[:80000], cmap='viridis_r')
plt.colorbar(sc)
plt.savefig(fname='graph_hidden-graph_size.png',dpi=600)

## max_pooling-future_growth

In [ ]:
test_trues.shape,output_pool.reshape(-1,64).shape

In [ ]:
plt.close()
Hs,Is = [],[]
for o,t,l in zip(output_pool.reshape(-1,64),test_trues,last_moment):
    if l < 3:
        continue
    # if t>8:
    #     continue
    # if t<4:
    #     continue
    Hs.append(o)
    Is.append(t)
Hs=np.array(Hs)
Hs,Is = shuffle(Hs,Is)
print(Hs.shape)
print(np.max(Is))
plt.close()
invisible(plt.gca())
X = TSNE(n_components=2,random_state=42,learning_rate=10,n_iter=6000,verbose=1).fit_transform(Hs[:10000])
sc = plt.scatter(X[:, 0], X[:, 1],s=1, c=Is[:10000], cmap='viridis_r')
plt.colorbar(sc)
plt.savefig(fname='max_pooling-future_growth.png',dpi=600)

In [ ]:
inc = []
for b in test_graph_size:
    inc.append(np.power(2,b[1:])-np.power(2,b[:-1]))
diff_hidden = []
for h in test_graph_hidden.reshape(-1,n_seq,hidden_dim):
    diff_hidden.append(h[1:]-h[:-1])
inc = np.array(inc)
diff_hidden = np.array(diff_hidden)
print(inc.shape,diff_hidden.shape)
print(inc.min(),inc.max())

In [ ]:
dHs = []
dIs = []
for h,i in zip(diff_hidden.reshape(-1,hidden_dim),inc.reshape(-1)):
    if i<2:
        continue
    dHs.append(h)
    dIs.append(np.log2(i))

print(len(dHs))

## difference_hidden-diff_size

In [ ]:
plt.close()
dHs,dIs = shuffle(dHs,dIs)
plt.close()
invisible(plt.gca())
X = TSNE(n_components=2,random_state=42,learning_rate=25,n_iter=3000,verbose=1).fit_transform(dHs[:10000])
sc = plt.scatter(X[:, 0], X[:, 1],s=1, c=dIs[:10000], cmap='viridis_r')
plt.colorbar(sc)
# 效果不错，数据有些少
plt.savefig(fname='difference-diff_size.png',dpi=600)

In [ ]:
# from sklearn.neural_network import MLPRegressor
# d_hidden, diff_inc=diff_hidden.reshape(-1,hidden_dim),inc.reshape(-1)
# model_mlp = MLPRegressor(
#     hidden_layer_sizes=(hidden_dim,1),  activation='relu', solver='adam', alpha=0.0001, batch_size='auto',
#     learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=50, shuffle=True,
#     random_state=1, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
#     early_stopping=False,beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# model_mlp.fit(d_hidden, diff_inc)

In [ ]:
# mlp_pre=model_mlp.predict(d_hidden)
# print(mlp_pre,diff_inc)
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# from sklearn.metrics import explained_variance_score
# mean_squared_error(mlp_pre, diff_inc),r2_score(mlp_pre, diff_inc),explained_variance_score(mlp_pre, diff_inc)